In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

from tqdm import tqdm

In [3]:
torch.device('cuda')

device(type='cuda')

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output


In [5]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [13]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x


In [14]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        cross_attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(cross_attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [15]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, device):
        super(Transformer, self).__init__()
        self.device = device
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(self.device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(self.device)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask.to(self.device)

        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [16]:
src_vocab_size = 500000
tgt_vocab_size = 500000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 50
dropout = 0.1
device = "cpu"

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout,  device)
transformer = transformer.to(device)

In [17]:
# start_token = torch.tensor([1999], dtype=torch.int64)

In [18]:
from random import randint

In [21]:
def generate_sample_10():
    # start^3, (start+1)^3 ...    start = randint(1, 500)
    start = 1
    extra = 10
    src = torch.tensor([i**2 for i in range(start, start+max_seq_length+extra)], dtype=torch.int64)
    trg = torch.tensor([(i+1)**2 for i in range(start, start+max_seq_length+extra)], dtype=torch.int64)
    labels = trg[1:]

    return src[:max_seq_length], trg[:max_seq_length], labels[:max_seq_length]


In [22]:
src_batch, trg_batch, labels = generate_sample_10()
print(src_batch, len(src_batch))
print(trg_batch, len(trg_batch))
print(labels, len(labels))

tensor([   1,    4,    9,   16,   25,   36,   49,   64,   81,  100,  121,  144,
         169,  196,  225,  256,  289,  324,  361,  400,  441,  484,  529,  576,
         625,  676,  729,  784,  841,  900,  961, 1024, 1089, 1156, 1225, 1296,
        1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304,
        2401, 2500]) 50
tensor([   4,    9,   16,   25,   36,   49,   64,   81,  100,  121,  144,  169,
         196,  225,  256,  289,  324,  361,  400,  441,  484,  529,  576,  625,
         676,  729,  784,  841,  900,  961, 1024, 1089, 1156, 1225, 1296, 1369,
        1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401,
        2500, 2601]) 50
tensor([   9,   16,   25,   36,   49,   64,   81,  100,  121,  144,  169,  196,
         225,  256,  289,  324,  361,  400,  441,  484,  529,  576,  625,  676,
         729,  784,  841,  900,  961, 1024, 1089, 1156, 1225, 1296, 1369, 1444,
        1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401

In [23]:
def generate_batch(batch_size: int = 128):
    src_batch = torch.tensor([], dtype=torch.int64)
    trg_batch = torch.tensor([], dtype=torch.int64)
    labels_batch = torch.tensor([], dtype=torch.int64)

    while src_batch.shape[0] < batch_size:
        src_sample, trg_sample, labels_sample = generate_sample_10()
        src_batch = torch.cat((src_batch, src_sample.unsqueeze(0)))
        trg_batch = torch.cat((trg_batch, trg_sample.unsqueeze(0)))
        labels_batch = torch.cat((labels_batch, labels_sample.unsqueeze(0)))

    return src_batch, trg_batch, labels_batch


In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()
train_loss_history = []

for step in tqdm(range(100)):
    src_batch, trg_batch, labels_batch = generate_batch(64)
    optimizer.zero_grad()
    output = transformer(src_batch.to(device), trg_batch.to(device))
    loss = criterion(output.view(-1, output.size(-1)), labels_batch.to(device).view(-1))
    loss.backward()
    optimizer.step()
    train_loss_history.append(loss.item())
    print(f"Step: {step+1}, Loss: {loss.item()}")

  0%|          | 0/100 [02:34<?, ?it/s]


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6400000000 bytes.

# Visualize train loss

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define X and Y variable data
x = np.array(list(range(len(train_loss_history))))
y = np.array(train_loss_history)

plt.plot(x, y)
plt.xlabel("Steps")  # add X-axis label
plt.ylabel("Train loss")  # add Y-axis label
plt.title("Training")  # add title
plt.show()

# Make batches for inference

In [ ]:
transformer.eval()
src_batch, trg_batch, labels_batch = generate_batch(64)

In [ ]:
preds_batch = torch.zeros_like(labels_batch)

In [ ]:
# predict from first token
trg_batch[:,1:] = torch.full(trg_batch[:,1:].shape, 0)

# Inference

In [ ]:
# generating cycle
for i in range(trg_batch.shape[1]):
    res = transformer(src_batch.to(device), trg_batch.to(device))
    preds_batch[:,i] = res[:,i].argmax(dim=1)

    if i < trg_batch.shape[1]-1:
        trg_batch[:,i+1] = res[:,i].argmax(dim=1)

In [ ]:
preds_batch

In [ ]:
trg_batch

# Compare labels and preds here

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
preds = preds_batch.view(-1).cpu().numpy()
labels = labels_batch.view(-1).cpu().numpy()

accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds, average='weighted')
recall = recall_score(labels, preds, average='weighted')
f1 = f1_score(labels, preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
